In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
import xgboost as XGB
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix,classification_report

%matplotlib inline


In [3]:
#Read csv file
train = pd.read_csv('Data/train.csv')
test = pd.read_csv('Data/test.csv')

In [4]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


### Exploratory Data Analysis

In [6]:
#Explore Datatypes of variables
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [7]:
#Check NULL VALUES
train.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [8]:
# Statistics proved that Cabin can be useful
train[train['Cabin'].isna()]['Survived'].value_counts()

0    481
1    206
Name: Survived, dtype: int64

In [9]:
train[train['Age'].isna()]['Survived'].value_counts()

0    125
1     52
Name: Survived, dtype: int64

In [10]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [11]:
#Removal of Visually Useless Columns 
def remove_col(df) -> pd.DataFrame():
    df = df.drop(['PassengerId','Name','Ticket'], axis = 1)
    return df


train_upd = remove_col(train)
train_upd.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,7.2500,NaN,S
1,1,1,female,38.0,1,0,71.2833,C85,C
2,1,3,female,26.0,0,0,7.9250,NaN,S
3,1,1,female,35.0,1,0,53.1000,C123,S
4,0,3,male,35.0,0,0,8.0500,NaN,S


In [12]:
train_upd.nunique()

Survived      2
Pclass        3
Sex           2
Age          88
SibSp         7
Parch         7
Fare        248
Cabin       147
Embarked      3
dtype: int64

In [13]:
def preprocessing(df,col) -> pd.DataFrame():
    #One Hot-Encoding
    
    df1 = pd.get_dummies(df[col], drop_first=True)
    df = df.join(df1)
    df.drop(col, axis = 1, inplace= True)
    return df

cols_for_onehot_enc = ['Sex','Embarked']
train_preprocess = preprocessing(train_upd, cols_for_onehot_enc)
train_preprocess.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Cabin,Sex_male,Embarked_Q,Embarked_S
0,0,3,22.0,1,0,7.2500,NaN,1,0,1
1,1,1,38.0,1,0,71.2833,C85,0,0,0
2,1,3,26.0,0,0,7.9250,NaN,0,0,1
3,1,1,35.0,1,0,53.1000,C123,0,0,1
4,0,3,35.0,0,0,8.0500,NaN,1,0,1


In [14]:
#Dealing with missing values
import sklearn.neighbors._base
import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

def missing_value(df) -> pd.DataFrame():
    
    df['Cabin'] = df['Cabin'].fillna(value = 0)
    df['Cabin'] = np.where(df['Cabin']!=0 , 1,0)
    
    imputer = MissForest()
    
    #df_new = df.drop('Survived', axis =1)
    X_imputed = imputer.fit_transform(df)
    df['Age'] = X_imputed[:,1]
    df['Fare'] = X_imputed[:,4]
    return df
    
final_X = missing_value(train_preprocess.drop('Survived', axis=1))
final_X.head()

Iteration: 0
Iteration: 1
Iteration: 2


,Pclass,Age,SibSp,Parch,Fare,Cabin,Sex_male,Embarked_Q,Embarked_S
0,3,22.0,1,0,7.2500,0,1,0,1
1,1,38.0,1,0,71.2833,1,0,0,0
2,3,26.0,0,0,7.9250,0,0,0,1
3,1,35.0,1,0,53.1000,1,0,0,1
4,3,35.0,0,0,8.0500,0,1,0,1


#### Note: For Alogrithms Missing values might be a problem but not for all algos. We will see

In [15]:
#Logit from statmodelsabs

final_X.dropna(inplace= True)

X = final_X.copy()
Y = train['Survived']

x = sm.add_constant(X)
Result = sm.Logit(Y,x).fit()
Result.summary()

Optimization terminated successfully.
         Current function value: 0.435831
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               Survived   No. Observations:                  891
Model:                          Logit   Df Residuals:                      881
Method:                           MLE   Df Model:                            9
Date:                Tue, 04 Oct 2022   Pseudo R-squ.:                  0.3455
Time:                        23:47:11   Log-Likelihood:                -388.33
converged:                       True   LL-Null:                       -593.33
Covariance Type:            nonrobust   LLR p-value:                 1.004e-82
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.8766      0.636      7.666      0.000       3.630       6.123
Pclass        -0.9688      0.178     -5.456      0.000      -1.317      -0.621
Age           -0.0423      0.008     -5.330      0.000      -0.058      -0.027
SibSp         -0.3649      0.110     -3.320      0.001      -0.580      -0.149
Parch         -0.0930      0.120     -0.778      0.436      -0.327       0.141
Fare           0.0008      0.002      0.325      0.745      -0.004       0.005
Cabin          0.6739      0.295      2.281      0.023       0.095       1.253
Sex_male      -2.6578      0.202    -13.188      0.000      -3.053      -2.263
Embarked_Q    -0.0703      0.395     -0.178      0.859      -0.844       0.703
Embarked_S    -0.3656      0.238     -1.533      0.125      -0.833       0.102
==============================================================================
"""

In [16]:
lg_model = LogisticRegression(penalty="l2", solver='lbfgs', C=100)
lg_model.fit(X,Y)
print('Score : ', lg_model.score(X,Y))
pred = lg_model.predict(X)
print(confusion_matrix(Y,pred))

Score :  0.8092031425364759
[[474  75]
 [ 95 247]]


In [17]:
#For Test

df1 = remove_col(test)
df2 = preprocessing(df1, cols_for_onehot_enc)
X_test = missing_value(df2)

Pred = lg_model.predict(X_test)

results = pd.DataFrame()
results['PassengerId'] = test['PassengerId']
results['Survived'] = Pred

results.to_csv('submission.csv', index= False)

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3


In [18]:
results

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
